Credentials

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

OPENAI_API_KEY=os.getenv('AZURE_OPENAI_API_KEY_US')
OPENAI_API_KEY_E=os.getenv('AZURE_OPENAI_API_KEY_US2') 

# os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2024-08-01-preview'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://azure-chat-try-2.openai.azure.com/'
os.environ['AZURE_OPENAI_DEPLOYMENT'] = 'chat-endpoint-us-gpt4o'

os.environ['OPENAI_API_VERSION_E'] = '2024-12-01-preview'
os.environ['AZURE_OPENAI_ENDPOINT_E'] = 'https://agents-4on.openai.azure.com/'
os.environ['AZURE_OPENAI_EMBEDDING_DEPLOYMENT_E'] = "text-embedding-3-large-eus2"

# LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
# os.environ['LANGCHAIN_TRACING_V2'] = 'true'
# os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
# os.environ['LANGCHAIN_PROJECT'] = "rag-sql"


LLM

In [2]:

from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

llm = AzureChatOpenAI(
    api_key = OPENAI_API_KEY,  
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
)

emb_model = AzureOpenAIEmbeddings(
    api_key=OPENAI_API_KEY_E,
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT_E'),  
    api_version=os.getenv('OPENAI_API_VERSION_E'),
    azure_deployment=os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_E')
)

Database

In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///./database/credit-risk.db", sample_rows_in_table_info=2)
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['collaterals', 'customers', 'sectors', 'transactions']


State

In [4]:
from langgraph.graph.message import MessagesState

class State(MessagesState):
    retrieved_docs: str
    query: str
    result: str
    answer: str

### Nodes

RAG

In [5]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="risk_db_tables",
    embedding_function=emb_model,
    persist_directory="./vector_db"
)

In [ ]:
# Check retrieval

# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough

# retriever = vector_store.as_retriever()

# template = """write an sql query that answers the question and uses the rertrieved database info:
# {context}

# Question: {question}
# """

# prompt = ChatPromptTemplate.from_template(template)

# rag_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# res = rag_chain.invoke("What is the total drawn exposure per corporate rating model and in the Agriculture sector for September 2023?")
# print(res)

# print('---------------------------')

# docs = retriever.vectorstore.similarity_search('What is the total off-balance exposure for September 2023?')
# print(docs)

Node: Retriever

In [6]:
from langgraph.types import Command

def db_retriever(state: State) -> Command[State]:

    msg = state['messages']
    question_message = msg[-1]
    question = question_message.content 

    docs = vector_store.similarity_search(question, k=5)

    snippets = []
    for d in docs:
        src = d.metadata.get("source", "unknown")
        snippets.append(f"Source: {src}\n{d.page_content}")

    return Command(
        goto="write_query",
        update={
            "retrieved_docs": snippets
        }
    )

Node: Convert question (+ schema) to SQL query (write query)

In [7]:
# from typing_extensions import Annotated
# from typing import Any, Dict, List, TypedDict
from langchain_core.messages import HumanMessage, AIMessage

write_query_instructions = """
You are a SQL expert with a strong attention to detail.

Given an input question, create a syntactically correct SQLite query to run to help find the answer. 

When generating the query:

You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the retrieved database description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

OUTPUT ONLY THE SQL QUERY!

Only use the following tables:
{retrieved_docs}

Question: {question}
"""

def write_query(state: State) -> Command[State]:
    """Generate SQL query to fetch information."""

    msg = state['messages']
    # ensure you're getting the content of the message for the prompt
    question_content = msg[-1].content
    retrieved_docs = state["retrieved_docs"]
    
    prompt = write_query_instructions.format(
        question = question_content, 
        retrieved_docs = retrieved_docs        
    ) 

    result = llm.invoke(prompt)

    return Command(
        update={
            # ensure the response content is stored as an AIMessage
            "messages": state['messages'] + [AIMessage(content=result.content.strip())],
            "query": result.content.strip()
        }
    )

Node: Execute query

In [8]:
def run_query(state: State) -> Command[State]:

    query = state['query']
    result = db.run_no_throw(query)

    return Command(
        update={
            "result": str(result)
        }
    )

Node: Generate answer

In [ ]:
answer_instructions = """
Given the user {question} and the query {result} compose a short but comprehensive final answer. 

If appropriate, present the final result in a tabluar form.
"""

def generate_answer(state: State) -> Command[State]:

    msg = state['messages']
    # ensure you're getting the content of the message for the prompt
    question_content = msg[-1].content
    result = state["result"]  

    prompt = answer_instructions.format(
        question=question_content, 
        result=result
    )   

    response = llm.invoke(prompt) 

    return Command(
            update={
                "messages": state['messages'] + [AIMessage(content=response.content)]
            }
    )

#### Flow

In [10]:
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display

builder = StateGraph(State)

builder.add_node('extract_db_info', db_retriever)
builder.add_node('write_query', write_query)
builder.add_node('run_query', run_query)
builder.add_node('generate_answer', generate_answer)

# builder.add_edge(START, 'extract_db_info')
builder.set_entry_point("extract_db_info")
builder.add_edge('extract_db_info', 'write_query')
builder.add_edge('write_query', 'run_query')
builder.add_edge('run_query', 'generate_answer') 
builder.add_edge('generate_answer', END)

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)
# display(Image(graph.get_graph().draw_mermaid_png()))

In [11]:
from langchain_core.messages import HumanMessage
from utils import format_messages

question_text = 'What is the total drawn exposure for each corporate rating model and in the Agriculture sector for September 2023?'

config = {"configurable": {"thread_id": "1"}} 

# pass the question as a HumanMessage in the 'messages' list
result = graph.invoke({"messages": [HumanMessage(content=question_text)]}, config=config)
format_messages(result['messages']) 

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ What is the total drawn exposure for each corporate rating model and in the Agriculture sector for September    │
│ 2023?                                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ ```sql                                                                                                          │
│ SELECT                                                                                                          │
│     c.RATING_MODEL,                                                                                             │
│     SUM(t.EXPOSURE) AS total_drawn_exposure                                                                     │
│ FROM                                                                                                            │
│     transactions t                                                                                              │
│ JOIN                                                                                                            │
│     customers c ON t.REF_DATE = c.REF_DATE AND t.PARTNER_ID = c.PARTNER_ID                                      │
│ JOIN                                                                                                            │
│     sectors s ON c.NACE = s.NACE                                                                                │
│ WHERE                                                                                                           │
│     t.PROCESSING = 'CE'                                                                                         │
│     AND c.RATING_MODEL IN ('CBIB10', 'CJHB10', 'CJIB10', 'FJHB10', 'FKBG20', 'FKBG21', 'FKBG30', 'FKCE11')      │
│     AND s.SECTOR = 'Agriculture, Forestry'                                                                      │
│     AND c.REF_DATE BETWEEN '01-09-2023' AND '30-09-2023'                                                        │
│ GROUP BY                                                                                                        │
│     c.RATING_MODEL                                                                                              │
│ ORDER BY                                                                                                        │
│     total_drawn_exposure DESC;                                                                                  │
│ ```                                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The error you are encountering is due to incorrect placement of the SQL code within triple backticks (```)      │
│ which are markdown syntax and not meant to be part of SQL execution. When using SQL outside of a markdown       │
│ editor, you should write the SQL code without the surrounding triple backticks as follows:                      │
│                                                                                                                 │
│ ```sql                                                                                                          │
│ SELECT                                                                                                          │
│     c.RATING_MODEL,                                                                                             │
│     SUM(t.EXPOSURE) AS total_drawn_exposure                                                                     │
│ FROM                                                                                                            │
│     transactions t                                                                                              │
│ JOIN                                                                                                            │
│     customers c ON t.REF_DATE = c.REF_DATE AND t.PARTNER_ID = c.PARTNER_ID                                      │
│ JOIN                                                                                                            │
│     sectors s ON c.NACE = s.NACE                                                                                │
│ WHERE                                                                                                           │
│     t.PROCESSING = 'CE'                                                                                         │
│     AND c.RATING_MODEL IN ('CBIB10', 'CJHB10', 'CJIB10', 'FJHB10', 'FKBG20', 'FKBG21', 'FKBG30', 'FKCE11')      │
│     AND s.SECTOR = 'Agriculture, Forestry'                                                                      │
│     AND c.REF_DATE BETWEEN '2023-09-01' AND '2023-09-30'  -- Ensure date format is consistent with your SQL     │
│ dialect                                                                                                         │
│ GROUP BY                                                                                                        │
│     c.RATING_MODEL                                                                                              │
│ ORDER BY                                                                                                        │
│     total_drawn_exposure DESC;                                                                                  │
│ ```                                                                                                             │
│                                                                                                                 │
│ Key corrections include:                                                                                        │
│                                                                                                                 │
│ 1. Remove any triple backticks from the SQL statement as they are not necessary when executing directly in a    │
│ SQL environment.                                                                                                │
│ 2. Verify that the date format matches that required by your SQL dialect, e.g., the format 'YYYY-MM-DD'.        │
│                                                                                                                 │
│ Run the corrected query in your SQL environment to get the desired results. The output will be a table with two │
│ columns: `RATING_MODEL` and `total_drawn_exposure`, sor